# 프로젝트 개요

1. 영화리뷰 데이터를 활용해 감성 분석을 진행
2. 코로나 관련 기사 제목 데이터를 활용해 감성 분석을 진행

* 감성분석 -> 긍정 / 부정 판단

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


# Library & Data Import

In [ ]:
import pandas as pd 
pd.__version__

'1.1.5'

In [ ]:
import warnings

warnings.filterwarnings(action='ignore')

In [ ]:
train_df = pd.read_csv('/content/gdrive/MyDrive/big_data/영화리뷰/data/ratings_train.txt', encoding="utf8", sep="\t")
train_df

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [ ]:
test_df = pd.read_csv('/content/gdrive/MyDrive/big_data/영화리뷰/data/ratings_test.txt', encoding="utf8", sep="\t")
test_df

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


# 데이터셋 살펴보기

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [ ]:
train_df.isnull().sum()

id          0
document    5
label       0
dtype: int64

In [ ]:
test_df.isnull().sum()

id          0
document    3
label       0
dtype: int64

In [ ]:
test_df['label'].value_counts()

1    25173
0    24827
Name: label, dtype: int64

In [ ]:
train_df =train_df.dropna()

## 정규표현식 적용
한글만 필터링해주기

In [ ]:
import re 

In [ ]:
train_df['document'] = train_df['document'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', "", x))
train_df

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1
...,...,...,...
149995,6222902,인간이 문제지 소는 뭔죄인가,0
149996,8549745,평점이 너무 낮아서,1
149997,9311800,이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다,0
149998,2376369,청춘 영화의 최고봉방황과 우울했던 날들의 자화상,1


In [ ]:
test_df = test_df[test_df['document'].notnull()]
test_df

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


In [ ]:
test_df['document'] = test_df['document'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', "", x))
test_df

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,,0
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따,0
49997,9072549,그림도 좋고 완성도도 높았지만 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화 재미도 없고 기분만 잡치고 한 세트장에서 다 해먹네,0


In [ ]:
# !pip install konlpy

     |████████████████████████████████| 19.4 MB 427 kB/s 
     |████████████████████████████████| 86 kB 4.9 MB/s 
     |████████████████████████████████| 448 kB 53.5 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


# vectorized

In [ ]:
from konlpy.tag import Okt

In [ ]:
okt=Okt()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer(tokenizer=okt.morphs, ngram_range=(1,2), min_df=3)

In [ ]:
tfidf.fit(train_df['document'])

TfidfVectorizer(min_df=3, ngram_range=(1, 2),
                tokenizer=<bound method Okt.morphs of <konlpy.tag._okt.Okt object at 0x7f7db74fd810>>)

In [ ]:
train_tfidf_df = tfidf.transform(train_df['document'])

In [ ]:
train_tfidf_df

<149995x115612 sparse matrix of type '<class 'numpy.float64'>'
	with 2703859 stored elements in Compressed Sparse Row format>

# 모델 학습

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression()

In [ ]:
lr.fit(train_tfidf_df, train_df['label'])

LogisticRegression()

In [ ]:
# test -> transform  // fit은 train data로!!!

test_tfidf_df = tfidf.transform(test_df['document'])

# 예측

In [ ]:
pred = lr.predict(test_tfidf_df)

# 평가

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print('감성 분석 정확도 : ', round(accuracy_score(test_df['label'], pred), 3))

감성 분석 정확도 :  0.852


In [ ]:
sen = input('감성 분석 단어 입력 >> ')

감성 분석 단어 입력 >> 기분좋다


In [ ]:
sen_re = re.sub(r'[^ ㄱ-ㅎ|가-힣]+', "", sen)

In [ ]:
sen_tfidf_df = tfidf.transform([sen_re])

In [ ]:
sen_pred = lr.predict(sen_tfidf_df)

In [ ]:
if (sen_pred[0]==0) : 
  print('부정감성')
else : 
  print('긍정감성')

긍정감성


# 코로나

In [ ]:
import json 

with open('/content/gdrive/MyDrive/big_data/영화리뷰/data/코로나_naver_news.json', encoding = 'utf8') as j_f : 
  json_data = json.load(j_f)

json_df = pd.DataFrame(json_data)

In [ ]:
json_df

,cnt,description,pDate,title
0,1,"<b>코로나</b>발 경제 위기 대응을 위해 돈 쓸 곳은 늘어났지만, 국세 수입은 ...",2020-06-04 14:12:00,결국 '증세론' 먼저 꺼내든 與…&quot;증세없는 '기본소득' 불가능&quot;
1,2,▲ 지난 2일 창녕군보건소 앞에 설치한 선별진료소에서 검사자가 체온을 측정하고 있다...,2020-06-04 14:12:00,"창녕군, ‘창녕형’비대면 선별진료소 운영"
2,3,"한편, ‘설현’은 최근 <b>코로나</b>19 바이러스를 다룬, MBC ‘시리즈M’...",2020-06-04 14:12:00,"DK모바일, 메인 홍보 모델로 AOA '설현' 선정"
3,4,변화의 핵심 중에서는 우리 사회가 신종 <b>코로나</b>바이러스 감염증(<b>코로...,2020-06-04 14:12:00,김병민 &quot;기본소득도 필요하면 논의 테이블에 올려야&quot; [인터뷰]
4,5,이재갑 고용노동부 장관은 6월3일(수) 11시에 <b>코로나</b>19 긴급 고용...,2020-06-04 14:12:00,"이재갑 장관, 고용안정지원금 서울센터 방문"
...,...,...,...,...
795,796,글로벌기업 현대.기아차가 <b>코로나</b>19의 여파로 해외시장에서 고전을 면치 ...,2020-06-04 13:36:00,"[세계의 눈] &quot;<b>코로나</b>19 영향, 현대차 매출 큰 타격&quo..."
796,797,한국중부발전 본사 전경 한국중부발전(사장 박형구)은 <b>코로나</b>19 장기화로...,2020-06-04 13:36:00,"중부발전, 소규모 태양광사업자 REC 판매대금 '선 지급'"
797,798,박종환 한국자유총연맹 총재 신종 <b>코로나</b>바이러스 감염증(<b>코로나</b...,2020-06-04 13:36:00,"&quot;<b>코로나</b> 함께 극복&quot;…박종환 자유총연맹 총재, 캠페인 참여"
798,799,이상훈 회장은 “<b>코로나</b>19 신규 확진자가 수도권에서 발생했고 정부는 수...,2020-06-04 13:36:00,치과의사 7000명 모이는 행사 앞두고 치협 회장 “행사 취소해달라”


In [ ]:
json_df = json_df[json_df['title'].notnull()]
# json_df

In [ ]:
json_df['title']= json_df['title'].apply(lambda x : re.sub(r'[^ ㄱ-ㅣ가-힣]+', " ", x))
json_df

,cnt,description,pDate,title
0,1,"<b>코로나</b>발 경제 위기 대응을 위해 돈 쓸 곳은 늘어났지만, 국세 수입은 ...",2020-06-04 14:12:00,결국 증세론 먼저 꺼내든 증세없는 기본소득 불가능
1,2,▲ 지난 2일 창녕군보건소 앞에 설치한 선별진료소에서 검사자가 체온을 측정하고 있다...,2020-06-04 14:12:00,창녕군 창녕형 비대면 선별진료소 운영
2,3,"한편, ‘설현’은 최근 <b>코로나</b>19 바이러스를 다룬, MBC ‘시리즈M’...",2020-06-04 14:12:00,모바일 메인 홍보 모델로 설현 선정
3,4,변화의 핵심 중에서는 우리 사회가 신종 <b>코로나</b>바이러스 감염증(<b>코로...,2020-06-04 14:12:00,김병민 기본소득도 필요하면 논의 테이블에 올려야 인터뷰
4,5,이재갑 고용노동부 장관은 6월3일(수) 11시에 <b>코로나</b>19 긴급 고용...,2020-06-04 14:12:00,이재갑 장관 고용안정지원금 서울센터 방문
...,...,...,...,...
795,796,글로벌기업 현대.기아차가 <b>코로나</b>19의 여파로 해외시장에서 고전을 면치 ...,2020-06-04 13:36:00,세계의 눈 코로나 영향 현대차 매출 큰 타격 하지만 차량은 순항
796,797,한국중부발전 본사 전경 한국중부발전(사장 박형구)은 <b>코로나</b>19 장기화로...,2020-06-04 13:36:00,중부발전 소규모 태양광사업자 판매대금 선 지급
797,798,박종환 한국자유총연맹 총재 신종 <b>코로나</b>바이러스 감염증(<b>코로나</b...,2020-06-04 13:36:00,코로나 함께 극복 박종환 자유총연맹 총재 캠페인 참여
798,799,이상훈 회장은 “<b>코로나</b>19 신규 확진자가 수도권에서 발생했고 정부는 수...,2020-06-04 13:36:00,치과의사 명 모이는 행사 앞두고 치협 회장 행사 취소해달라


In [ ]:
json_tfidf = tfidf.transform(json_df['title'])

In [ ]:
pred_news = lr.predict(json_tfidf)

In [ ]:
n_cnt = 0
p_cnt =0 

for item in pred_news :
  if item == 0 : 
    n_cnt += 1
  else :
    p_cnt +=1 
print('긍정 :' ,p_cnt, "\n부정 :", n_cnt)

긍정 : 281 
부정 : 519
